# Call Fan Discovery

Build a **fan** of BTO calls across all available expirations (weeklies through LEAPS) for your conviction tickers.

**How to use:** Edit the config cell below, then **Run All**.

1. Set your **tickers and strike zones** (cell 3)
2. Set your **portfolio size** (cell 3)
3. Run all -- get a **BUY LIST** you can execute


In [13]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

from notebook_pipeline import (
    setup_report_style,
    display_table,
    fetch_underlying_metrics,
    fetch_fan_candidates,
    score_option_candidates,
    build_best_fan,
    score_fan,
    bs_call_greeks,
    build_correlation_matrix,
)
from notebook_reporting import export_report_bundle

setup_report_style()

FIGURE_COUNTER = 0


def show_figure(fig, title):
    global FIGURE_COUNTER
    FIGURE_COUNTER += 1
    if not str(title).lower().startswith("figure"):
        title = f"Figure {FIGURE_COUNTER}. {title}"
    fig.update_layout(title=title)
    fig.show()

In [14]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  EDIT THIS CELL — your tickers, strike zones, and budget   ║
# ╚══════════════════════════════════════════════════════════════╝

# Your tickers and strike zones (low, high).
# Use None instead of a range to auto-detect based on current price.
MY_TICKERS = {
    "CMI": (500, 600),
    "SNPS": (415, 440),
    "GEV": (750, 800),
}

# Your total portfolio budget
MY_BUDGET = 15_000

# Max legs per fan (how many expirations to include per ticker)
MAX_LEGS = 8

# ──────────────────────────────────────────────────────────────
# Advanced settings (you can ignore these)
# ──────────────────────────────────────────────────────────────
DEFAULT_MONEYNESS = (0.85, 1.10)  # fallback if no strike range set
MIN_DTE = 3  # earliest expiration (days out)
MAX_DTE = 760  # furthest expiration (days out)
RATE_LIMIT_SLEEP = 0.25  # pause between API calls (seconds)

# ──────────────────────────────────────────────────────────────
FAN_TICKERS = MY_TICKERS
STARTING_BALANCE = float(MY_BUDGET)
MAX_LEGS_PER_FAN = MAX_LEGS
EXPORT_EXCEL = True
EXPORT_ZIP = True
RUN_STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = "outputs"

display(Markdown("### Your Setup"))
display(Markdown(f"**Budget:** ${STARTING_BALANCE:,.0f}"))
for tkr, rng in FAN_TICKERS.items():
    if rng:
        display(Markdown(f"- **{tkr}** strike zone: ${rng[0]:,.0f} -- ${rng[1]:,.0f}"))
    else:
        display(Markdown(f"- **{tkr}** (auto-detect strike zone)"))

### Your Setup

**Budget:** $15,000

- **CMI** strike zone: $500 -- $600

- **SNPS** strike zone: $415 -- $440

- **GEV** strike zone: $750 -- $800

In [15]:
# --- Fetch market data (just run this, no edits needed) ---
tickers = list(FAN_TICKERS.keys())

display(Markdown("Fetching underlying data ..."))
metrics_df = fetch_underlying_metrics(
    tickers,
    history_period="1y",
    rate_limit_sleep=RATE_LIMIT_SLEEP,
)
if metrics_df.empty:
    raise RuntimeError("No underlying metrics fetched.")

# Quick snapshot of current prices
for _, row in metrics_df.iterrows():
    rng = FAN_TICKERS.get(row["ticker"])
    zone = f"${rng[0]:,.0f}-${rng[1]:,.0f}" if rng else "auto"
    display(
        Markdown(
            f"**{row['ticker']}** spot ${row['spot']:,.2f}  |  "
            f"strike zone: {zone}  |  "
            f"3m return: {row['ret_3m']:+.1%}  |  HV: {row['hv_30']:.1%}"
        )
    )

# Fetch all option chains
display(Markdown("---"))
display(Markdown("Fetching option chains (this takes a minute) ..."))
fan_chains: dict[str, pd.DataFrame] = {}
for tkr, strike_rng in FAN_TICKERS.items():
    row = metrics_df[metrics_df["ticker"] == tkr]
    if row.empty:
        continue
    spot = float(row.iloc[0]["spot"])
    df = fetch_fan_candidates(
        tkr,
        spot=spot,
        strike_range=strike_rng,
        moneyness_range=DEFAULT_MONEYNESS,
        min_dte=MIN_DTE,
        max_dte=MAX_DTE,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
    )
    if df.empty:
        display(Markdown(f"**{tkr}**: no contracts found in strike zone"))
    else:
        display(
            Markdown(
                f"**{tkr}**: found {len(df)} contracts across "
                f"{df['expiration'].nunique()} expirations"
            )
        )
        fan_chains[tkr] = df

if not fan_chains:
    raise RuntimeError("No fan chains fetched for any ticker.")
display(Markdown("**Done.**"))

Fetching underlying data ...

**CMI** spot $577.73  |  strike zone: $500-$600  |  3m return: +32.1%  |  HV: 44.6%

**SNPS** spot $426.88  |  strike zone: $415-$440  |  3m return: +4.3%  |  HV: 43.8%

**GEV** spot $779.35  |  strike zone: $750-$800  |  3m return: +39.3%  |  HV: 42.7%

---

Fetching option chains (this takes a minute) ...

**CMI**: found 45 contracts across 5 expirations

**SNPS**: found 36 contracts across 11 expirations

**GEV**: found 93 contracts across 15 expirations

**Done.**

In [21]:
# --- Build fans and generate BUY LIST ---
fans: dict[str, pd.DataFrame] = {}
fan_metrics: dict[str, dict] = {}

for tkr, chain_df in fan_chains.items():
    tkr_metrics = metrics_df[metrics_df["ticker"] == tkr]
    spot = float(tkr_metrics.iloc[0]["spot"])
    scored = score_option_candidates(chain_df, tkr_metrics)
    if scored.empty:
        continue
    fan = build_best_fan(scored, max_legs=MAX_LEGS_PER_FAN)
    if fan.empty:
        continue
    fans[tkr] = fan
    fan_metrics[tkr] = score_fan(fan, spot)

# Score-weighted portfolio allocation
fan_summary_df = pd.DataFrame(
    [
        {
            "ticker": tkr,
            "legs": fm["n_legs"],
            "total_cost": fm["total_cost_100sh"],
            "fan_score": fm["fan_score"],
            "agg_delta": fm["agg_delta"],
            "agg_theta": fm["agg_theta"],
            "agg_vega": fm["agg_vega"],
            "total_premium": fm["total_premium"],
            "iv_term_slope": fm["iv_term_slope"],
        }
        for tkr, fm in fan_metrics.items()
    ]
).sort_values("fan_score", ascending=False)

port = fan_summary_df.copy()
total_score = port["fan_score"].sum()
port["weight"] = port["fan_score"] / total_score if total_score > 0 else 1 / len(port)
port["weight"] = port["weight"].clip(lower=0.10, upper=0.50)
port["weight"] = port["weight"] / port["weight"].sum()
port["dollar_alloc"] = port["weight"] * STARTING_BALANCE
port["fan_sets"] = (
    np.floor(port["dollar_alloc"] / port["total_cost"]).astype(int).clip(lower=1)
)
port["deployed"] = port["fan_sets"] * port["total_cost"]

# ╔══════════════════════════════════════════════════════════════╗
# ║                        BUY LIST                             ║
# ╚══════════════════════════════════════════════════════════════╝
display(Markdown("---"))
display(Markdown("# BUY LIST"))
display(Markdown(f"**Budget:** ${STARTING_BALANCE:,.0f}"))

buy_rows = []
warn_rows = []
order_num = 0

for _, p in port.iterrows():
    tkr = p["ticker"]
    fan = fans.get(tkr)
    if fan is None:
        continue
    n_sets = int(p["fan_sets"])
    spot = float(metrics_df[metrics_df["ticker"] == tkr].iloc[0]["spot"])
    hv = float(metrics_df[metrics_df["ticker"] == tkr].iloc[0]["hv_30"])
    rsi = float(metrics_df[metrics_df["ticker"] == tkr].iloc[0]["rsi_14"])
    ret_3m = float(metrics_df[metrics_df["ticker"] == tkr].iloc[0]["ret_3m"])

    display(
        Markdown(
            f"## {tkr}  ({int(p['legs'])} legs x {n_sets} set(s) = ${p['deployed']:,.0f})"
        )
    )

    for _, leg in fan.iterrows():
        order_num += 1
        mid = float(leg["mid"])
        strike = float(leg["strike"])
        dte = int(leg["dte"])
        iv = float(leg["iv"])
        oi = float(leg["open_interest"])
        spread_pct = float(leg["spread_pct"]) if not np.isnan(leg["spread_pct"]) else 0
        cost_per = mid * 100 * n_sets

        # Greeks for this leg
        g = bs_call_greeks(spot=spot, strike=strike, dte=dte, iv=iv)
        delta = g["delta"]
        theta = g["theta"]
        vega = g["vega"]

        # --- Sell targets ---
        breakeven = strike + mid
        breakeven_pct = breakeven / spot - 1
        profit_target_50 = mid * 1.50  # take profit at +50%
        profit_target_100 = mid * 2.00  # take profit at +100%
        stop_loss = mid * 0.50  # cut at -50% of premium

        # What stock price gets you to those targets (rough intrinsic)
        stock_at_50_gain = strike + profit_target_50
        stock_at_100_gain = strike + profit_target_100

        # --- Time decay warning ---
        # Days until 50% of premium erodes (linear approx)
        daily_theta_abs = abs(theta) if not np.isnan(theta) else 0
        days_to_half = int(mid * 0.5 / daily_theta_abs) if daily_theta_abs > 0 else 999
        theta_per_week = daily_theta_abs * 7

        # --- Warnings ---
        warnings = []
        if iv > hv * 1.5:
            warnings.append("IV is 1.5x historical vol -- you're overpaying for vol")
        if iv > 0.60:
            warnings.append("IV > 60% -- expensive premium, need a big move to profit")
        if spread_pct > 0.15:
            warnings.append(
                f"Wide spread ({spread_pct:.0%}) -- use limit orders, don't market-buy"
            )
        if oi < 50:
            warnings.append(f"Low open interest ({oi:.0f}) -- may be hard to exit")
        if dte <= 14:
            warnings.append("< 2 weeks to expiry -- theta is accelerating fast")
        if dte <= 7:
            warnings.append("EXPIRING THIS WEEK -- close or roll immediately")
        if breakeven_pct > 0.15:
            warnings.append(
                f"Stock needs to rally {breakeven_pct:+.0%} just to break even"
            )
        if rsi > 70:
            warnings.append(
                f"RSI is {rsi:.0f} -- overbought, consider waiting for a pullback"
            )
        if rsi < 30:
            warnings.append(
                f"RSI is {rsi:.0f} -- oversold, could be catching a falling knife"
            )
        if ret_3m < -0.10:
            warnings.append(
                f"Stock is down {ret_3m:.0%} over 3 months -- trend is against you"
            )

        # Priority flag
        if dte <= 21 and delta > 0.55:
            priority = "NOW"
        elif dte <= 45:
            priority = "SOON"
        else:
            priority = "PATIENT"

        buy_rows.append(
            {
                "#": order_num,
                "Action": "BUY TO OPEN",
                "Ticker": tkr,
                "Contract": leg["contract_symbol"],
                "Strike": strike,
                "Exp": leg["expiration"],
                "DTE": dte,
                "Price": mid,
                "Qty": n_sets,
                "Cost": cost_per,
                "IV": iv,
                "Delta": delta,
                "Breakeven": breakeven,
                "BE Dist": breakeven_pct,
                "Sell +50%": profit_target_50,
                "Sell +100%": profit_target_100,
                "Stop Loss": stop_loss,
                "Stock@+50%": stock_at_50_gain,
                "Stock@+100%": stock_at_100_gain,
                "Theta/wk": -theta_per_week,
                "Days to 50% decay": days_to_half,
                "Urgency": priority,
                "Warnings": "; ".join(warnings) if warnings else "",
            }
        )

buy_df = pd.DataFrame(buy_rows)

# --- Transaction Table ---
display_table(
    buy_df[
        [
            "#",
            "Action",
            "Ticker",
            "Contract",
            "Strike",
            "Exp",
            "DTE",
            "Price",
            "Qty",
            "Cost",
            "IV",
            "Delta",
            "Urgency",
        ]
    ],
    caption="Transactions",
    format_dict={
        "Strike": "${:,.2f}",
        "Price": "${:,.2f}",
        "Cost": "${:,.0f}",
        "IV": "{:.1%}",
        "Delta": "{:.2f}",
    },
)

# Bottom line
total_deployed = buy_df["Cost"].sum()
remaining = STARTING_BALANCE - total_deployed
display(Markdown("---"))
display(
    Markdown(
        f"**Total cost:** ${total_deployed:,.0f}  |  "
        f"**Cash remaining:** ${remaining:,.0f}  |  "
        f"**{len(buy_df)} orders across {len(fans)} tickers**"
    )
)

# ╔══════════════════════════════════════════════════════════════╗
# ║                     SELL TARGETS                            ║
# ╚══════════════════════════════════════════════════════════════╝
display(Markdown("---"))
display(Markdown("# SELL TARGETS & EXITS"))

display_table(
    buy_df[
        [
            "#",
            "Ticker",
            "Contract",
            "Exp",
            "Price",
            "Breakeven",
            "BE Dist",
            "Stop Loss",
            "Sell +50%",
            "Sell +100%",
            "Stock@+50%",
            "Stock@+100%",
        ]
    ],
    caption="When to Sell",
    format_dict={
        "Price": "${:,.2f}",
        "Breakeven": "${:,.2f}",
        "BE Dist": "{:+.1%}",
        "Stop Loss": "${:,.2f}",
        "Sell +50%": "${:,.2f}",
        "Sell +100%": "${:,.2f}",
        "Stock@+50%": "${:,.0f}",
        "Stock@+100%": "${:,.0f}",
    },
)

display(
    Markdown(
        "**How to read this:** For each contract, the table shows:\n"
        "- **Breakeven** -- stock price where you stop losing money at expiry\n"
        "- **BE Dist** -- how far the stock needs to move to break even\n"
        "- **Stop Loss** -- sell if premium drops to this (50% loss rule)\n"
        "- **Sell +50% / +100%** -- take-profit targets on the option price\n"
        "- **Stock@+50% / +100%** -- approximate stock price needed to hit those targets"
    )
)

# ╔══════════════════════════════════════════════════════════════╗
# ║                     WARNINGS                                ║
# ╚══════════════════════════════════════════════════════════════╝
display(Markdown("---"))
display(Markdown("# WARNINGS & ALERTS"))

flagged = buy_df[buy_df["Warnings"] != ""].copy()
if flagged.empty:
    display(Markdown("No warnings. All positions look clean."))
else:
    for _, row in flagged.iterrows():
        alerts = row["Warnings"].split("; ")
        display(Markdown(f"### {row['Ticker']} -- {row['Contract']}"))
        for alert in alerts:
            display(Markdown(f"- {alert}"))

# ╔══════════════════════════════════════════════════════════════╗
# ║                   TIME DECAY WATCH                          ║
# ╚══════════════════════════════════════════════════════════════╝
display(Markdown("---"))
display(Markdown("# TIME DECAY WATCH"))

display_table(
    buy_df[
        [
            "#",
            "Ticker",
            "Contract",
            "DTE",
            "Price",
            "Theta/wk",
            "Days to 50% decay",
            "Urgency",
        ]
    ].sort_values("Days to 50% decay"),
    caption="Theta Erosion Schedule",
    format_dict={
        "Price": "${:,.2f}",
        "Theta/wk": "${:,.2f}",
    },
)

# Calendar-style roll/close reminders
display(Markdown("### Action Calendar"))
for _, row in buy_df.sort_values("DTE").iterrows():
    dte = row["DTE"]
    if dte <= 7:
        action = "CLOSE OR ROLL THIS WEEK"
        icon = "!!"
    elif dte <= 21:
        action = "Set alerts -- theta accelerating"
        icon = "!"
    elif dte <= 45:
        action = "Monitor weekly, consider rolling at 21 DTE"
        icon = "~"
    elif dte <= 90:
        action = "Check monthly, roll if thesis changes"
        icon = ""
    else:
        action = "Sit tight, re-evaluate at 90 DTE"
        icon = ""
    display(Markdown(f"- **{icon} {row['Ticker']} {row['Exp']}** ({dte}d) -- {action}"))

# ╔══════════════════════════════════════════════════════════════╗
# ║                   POSITION RULES                            ║
# ╚══════════════════════════════════════════════════════════════╝
display(Markdown("---"))
display(Markdown("# POSITION MANAGEMENT RULES"))
display(
    Markdown(
        "| Rule | Action |\n"
        "|------|--------|\n"
        "| Premium drops 50% | **Sell.** Cut the loss, redeploy capital |\n"
        "| Premium doubles (+100%) | **Sell half.** Let the rest ride free |\n"
        "| Premium up 50% | **Sell a third** or tighten your mental stop to breakeven |\n"
        "| DTE hits 21 days | **Roll or close** short-dated legs (theta cliff) |\n"
        "| DTE hits 7 days | **Close.** Do not hold into expiration week unless deep ITM |\n"
        "| Stock gaps down > 5% | Re-evaluate thesis. If broken, close all legs on that ticker |\n"
        "| IV spikes > 20% | Consider selling -- you're being paid more for the same position |\n"
        "| IV crashes > 20% | Your vega is working against you -- tighten stops |\n"
        f"| Portfolio down > ${STARTING_BALANCE * 0.10:,.0f} | **Stop trading.** Step back and reassess |\n"
        f"| Portfolio up > ${STARTING_BALANCE * 0.30:,.0f} | Take some off -- don't give it all back |"
    )
)

---

# BUY LIST

**Budget:** $15,000

## CMI  (5 legs x 1 set(s) = $21,290)

## GEV  (8 legs x 1 set(s) = $36,400)

...

- ** CMI 2026-09-18** (222d) -- Sit tight, re-evaluate at 90 DTE

- ** SNPS 2026-09-18** (222d) -- Sit tight, re-evaluate at 90 DTE

---

# POSITION MANAGEMENT RULES

| Rule | Action |
|------|--------|
| Premium drops 50% | **Sell.** Cut the loss, redeploy capital |
| Premium doubles (+100%) | **Sell half.** Let the rest ride free |
| Premium up 50% | **Sell a third** or tighten your mental stop to breakeven |
| DTE hits 21 days | **Roll or close** short-dated legs (theta cliff) |
| DTE hits 7 days | **Close.** Do not hold into expiration week unless deep ITM |
| Stock gaps down > 5% | Re-evaluate thesis. If broken, close all legs on that ticker |
| IV spikes > 20% | Consider selling -- you're being paid more for the same position |
| IV crashes > 20% | Your vega is working against you -- tighten stops |
| Portfolio down > $1,500 | **Stop trading.** Step back and reassess |
| Portfolio up > $4,500 | Take some off -- don't give it all back |

---

## Deep Dive (optional)

Charts and scenario analysis below. Scroll past if you just need the buy list above.


In [17]:
# --- Key Charts (optional — scroll past if you just need the buy list) ---
if fans:
    all_fans = pd.concat(fans.values(), ignore_index=True)

    # 1. Premium Ladder — what you're paying at each expiration
    fig_prem = px.scatter(
        all_fans,
        x="dte",
        y="mid",
        color="ticker",
        size="open_interest",
        hover_data=["contract_symbol", "strike", "iv", "expiration"],
        height=450,
    )
    fig_prem.update_yaxes(title="Premium ($)", tickprefix="$")
    fig_prem.update_xaxes(title="Days to Expiration")
    show_figure(fig_prem, "What You're Paying (Premium vs DTE)")

    # 2. IV Term Structure — is vol cheap or expensive further out?
    fig_iv = px.line(
        all_fans,
        x="dte",
        y="iv",
        color="ticker",
        markers=True,
        height=400,
    )
    fig_iv.update_yaxes(tickformat=".0%", title="Implied Volatility")
    fig_iv.update_xaxes(title="Days to Expiration")
    show_figure(fig_iv, "IV Term Structure")

    # 3. Fan Score Ranking
    fig_rank = px.bar(
        fan_summary_df.sort_values("fan_score"),
        x="fan_score",
        y="ticker",
        orientation="h",
        color="fan_score",
        color_continuous_scale="Blues",
        height=max(250, len(fan_summary_df) * 70),
        text="fan_score",
    )
    fig_rank.update_traces(texttemplate="%{text:.1f}", textposition="outside")
    fig_rank.update_layout(coloraxis_showscale=False)
    fig_rank.update_xaxes(title="Fan Score")
    show_figure(fig_rank, "Fan Score Ranking")

In [18]:
# --- What-If Analysis: how do your fans perform if the stock moves? ---
if fan_metrics:
    # Scenario P&L bar chart (all tickers)
    scen_rows = []
    for tkr, fm in fan_metrics.items():
        for label, pnl in fm["scenario_pnl"].items():
            clean = (
                label.replace("_", " ")
                .replace("bear", "Bear")
                .replace("bull", "Bull")
                .replace("flat", "Flat")
            )
            scen_rows.append({"ticker": tkr, "scenario": clean, "pnl": pnl})

    fig_scen = px.bar(
        pd.DataFrame(scen_rows),
        x="ticker",
        y="pnl",
        color="scenario",
        barmode="group",
        height=450,
        color_discrete_sequence=["#B0533C", "#D4A574", "#8B9BB4", "#4C6E91", "#1F3A5F"],
    )
    fig_scen.update_yaxes(title="P&L per set ($)", tickprefix="$")
    show_figure(fig_scen, "Scenario P&L (per 1-contract set)")

    # 2D Sensitivity surface for the top-scoring fan
    top_tkr = fan_summary_df.iloc[0]["ticker"]
    top_fan = fans[top_tkr]
    top_spot = float(metrics_df[metrics_df["ticker"] == top_tkr].iloc[0]["spot"])

    spot_moves = np.arange(-0.25, 0.30, 0.05)
    iv_shocks = np.arange(-0.15, 0.20, 0.05)
    surface = np.zeros((len(iv_shocks), len(spot_moves)))

    for i, iv_shock in enumerate(iv_shocks):
        for j, spot_move in enumerate(spot_moves):
            total_pnl = 0.0
            for _, leg in top_fan.iterrows():
                terminal = top_spot * (1.0 + spot_move)
                intrinsic = max(terminal - float(leg["strike"]), 0.0)
                new_iv = max(float(leg["iv"]) + iv_shock, 0.05)
                half_dte = max(int(leg["dte"]) // 2, 1)
                if intrinsic > 0 and half_dte > 1:
                    g = bs_call_greeks(
                        spot=terminal,
                        strike=float(leg["strike"]),
                        dte=half_dte,
                        iv=new_iv,
                    )
                    time_val = (
                        max(g["vega"] * new_iv * 10, 0)
                        if not np.isnan(g["vega"])
                        else 0
                    )
                    leg_val = intrinsic + time_val
                else:
                    leg_val = intrinsic
                total_pnl += leg_val - float(leg["mid"])
            surface[i, j] = total_pnl

    fig_surf = go.Figure(
        data=go.Heatmap(
            z=surface,
            x=[f"{m:+.0%}" for m in spot_moves],
            y=[f"{s:+.0%}" for s in iv_shocks],
            colorscale="RdYlGn",
            text=np.round(surface, 1).astype(str),
            texttemplate="$%{text}",
            hovertemplate="Spot: %{x}<br>IV shock: %{y}<br>P&L: $%{z:,.0f}<extra></extra>",
        )
    )
    fig_surf.update_layout(
        height=450,
        xaxis_title="Stock Move",
        yaxis_title="IV Change",
    )
    show_figure(fig_surf, f"{top_tkr} What-If Surface (Stock Move x IV Change)")

In [ ]:
# --- Portfolio Risk Summary ---
if not port.empty:
    total_deployed = port["deployed"].sum()
    remaining = STARTING_BALANCE - total_deployed
    total_theta = (port["agg_theta"] * port["fan_sets"]).sum()
    total_delta = (port["agg_delta"] * port["fan_sets"]).sum()
    total_vega = sum(
        fan_metrics[tkr]["agg_vega"]
        * int(port[port["ticker"] == tkr]["fan_sets"].iloc[0])
        for tkr in fans
    )
    weekly_theta = total_theta * 7
    monthly_theta = total_theta * 30

    # Allocation donut
    fig_donut = go.Figure(
        data=[
            go.Pie(
                labels=port["ticker"],
                values=port["deployed"],
                hole=0.50,
                marker_colors=[
                    "#1F3A5F",
                    "#4C6E91",
                    "#B0533C",
                    "#D4A574",
                    "#6B8E6B",
                    "#8B9BB4",
                ],
                textinfo="label+percent",
            )
        ]
    )
    fig_donut.update_layout(
        height=420,
        annotations=[
            dict(
                text=f"${total_deployed:,.0f}",
                x=0.5,
                y=0.5,
                font_size=18,
                showarrow=False,
            )
        ],
    )
    show_figure(fig_donut, "Capital Allocation")

    # Concentration check
    max_weight = port["weight"].max()
    conc_warn = ""
    if max_weight > 0.40:
        conc_warn = f" -- **Concentration risk:** {port.iloc[0]['ticker']} is {max_weight:.0%} of portfolio"

    display(Markdown("### Risk at a Glance"))
    display(
        Markdown(
            f"| Metric | Value |\n"
            f"|--------|-------|\n"
            f"| Total deployed | ${total_deployed:,.0f} of ${STARTING_BALANCE:,.0f} ({total_deployed / STARTING_BALANCE:.0%}) |\n"
            f"| Cash reserve | ${remaining:,.0f} ({remaining / STARTING_BALANCE:.0%}) |\n"
            f"| **Max loss (all premiums)** | **${total_deployed:,.0f}** |\n"
            f"| Portfolio delta | {total_delta:.3f} |\n"
            f"| Portfolio vega | {total_vega:.3f} |\n"
            f"| Daily theta burn | ${total_theta:+.2f} |\n"
            f"| Weekly theta burn | ${weekly_theta:+.2f} |\n"
            f"| Monthly theta burn | ${monthly_theta:+.2f} |\n"
            f"| Fans | {len(port)} tickers, {int(port['legs'].sum())} total legs |"
        )
    )
    if conc_warn:
        display(Markdown(conc_warn))

    # Per-ticker risk scorecard
    display(Markdown("### Per-Ticker Risk Scorecard"))
    risk_cards = []
    for _, p in port.iterrows():
        tkr = p["ticker"]
        fm = fan_metrics[tkr]
        spot = float(metrics_df[metrics_df["ticker"] == tkr].iloc[0]["spot"])
        hv = float(metrics_df[metrics_df["ticker"] == tkr].iloc[0]["hv_30"])
        avg_iv = fans[tkr]["iv"].mean()
        soonest_dte = int(fans[tkr]["dte"].min())
        farthest_dte = int(fans[tkr]["dte"].max())

        # Risk score (lower = riskier)
        risk_flags = 0
        if avg_iv > hv * 1.3:
            risk_flags += 1
        if soonest_dte < 21:
            risk_flags += 1
        if p["weight"] > 0.35:
            risk_flags += 1
        if fm["iv_term_slope"] < -0.001:
            risk_flags += 1
        risk_level = ["LOW", "MODERATE", "ELEVATED", "HIGH", "CRITICAL"][
            min(risk_flags, 4)
        ]

        risk_cards.append(
            {
                "Ticker": tkr,
                "Weight": p["weight"],
                "Deployed": p["deployed"],
                "Avg IV": avg_iv,
                "HV 30d": hv,
                "IV/HV": avg_iv / hv if hv > 0 else 0,
                "Soonest Exp": f"{soonest_dte}d",
                "Farthest Exp": f"{farthest_dte}d",
                "Fan Score": fm["fan_score"],
                "Risk": risk_level,
            }
        )

    risk_df = pd.DataFrame(risk_cards)
    display_table(
        risk_df,
        caption="Risk Assessment",
        format_dict={
            "Weight": "{:.0%}",
            "Deployed": "${:,.0f}",
            "Avg IV": "{:.1%}",
            "HV 30d": "{:.1%}",
            "IV/HV": "{:.2f}",
            "Fan Score": "{:.2f}",
        },
    )

### Risk at a Glance

| Metric | Value |
|--------|-------|
| Total deployed | $87,760 of $15,000 (585%) |
| Cash reserve | $-72,760 (-485%) |
| **Max loss (all premiums)** | **$87,760** |
| Portfolio delta | 11.789 |
| Portfolio vega | 16.528 |
| Daily theta burn | $-13.02 |
| Weekly theta burn | $-91.12 |
| Monthly theta burn | $-390.49 |
| Fans | 3 tickers, 21 total legs |

### Per-Ticker Risk Scorecard

,Ticker,Weight,Deployed,Avg IV,HV 30d,IV/HV,Soonest Exp,Farthest Exp,Fan Score,Risk
0,CMI,37%,"$21,290",32.2%,44.6%,0.72,12d,222d,62.22,ELEVATED
1,GEV,36%,"$36,400",51.6%,42.7%,1.21,5d,68d,61.04,ELEVATED
2,SNPS,27%,"$30,070",56.7%,43.8%,1.29,5d,222d,45.96,MODERATE


In [20]:
# --- Export ---
export_frames: dict[str, pd.DataFrame] = {}

if fan_chains:
    export_frames["fan_chains"] = pd.concat(fan_chains.values(), ignore_index=True)
if fans:
    export_frames["best_fans"] = pd.concat(fans.values(), ignore_index=True)
if not fan_summary_df.empty:
    export_frames["fan_summary"] = fan_summary_df
if "buy_df" in dir() and not buy_df.empty:
    export_frames["buy_list"] = buy_df
if "port" in dir() and not port.empty:
    port_export = port.drop(
        columns=["scenario_pnl", "greeks_df", "scenario_df"], errors="ignore"
    )
    export_frames["portfolio"] = port_export

if export_frames:
    bundle = export_report_bundle(
        prefix="call_fan",
        run_stamp=RUN_STAMP,
        output_dir=OUTPUT_DIR,
        frames=export_frames,
        metadata={
            "notebook": "call_fan_discovery",
            "tickers": list(FAN_TICKERS.keys()),
            "max_legs_per_fan": MAX_LEGS_PER_FAN,
            "starting_balance": STARTING_BALANCE,
        },
        include_excel=EXPORT_EXCEL,
        include_zip=EXPORT_ZIP,
    )
    display(Markdown(f"**Exported to:** `{bundle['run_dir']}`"))
else:
    display(Markdown("Nothing to export."))

**Exported to:** `outputs/call_fan_20260208_180051`